<a href="https://colab.research.google.com/github/s-joshid/Transporter_proj/blob/main/FIXED_Best_annotationScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import argparse
#argparse makes it easy to write user friendly command-line interfaces
import os

Want to parse files by drawing out most important cols. This script is focused on finding the best annotations so need to pull out the column relating to that as well as identifiers.

from the HMMER doc (pg 33/227):
E-VAL: "the most important number is the E-value... it is the expected number of false positives (non-homologus sequences) that scored this well or better... [it] is a measure of satitical significance. the lower the E-value, the more dignificant the hit" (doc typically considers sequences w/ E-values< 10^-3 to be significant)

Score: 2nd num; what EVal is based off of. it's the log-odd score for complete sequence. doesnt depend on size of sequence DB, only on profile & target sequence (E val kinda does bc larger DBs means more false pos)

bias: correction term for biased sequence comp; can be over or under aggressive, leading to missed homologus sequences or allowing non-homolog through

next 3 nums are again E-val, score & bias, but these are for only for single-best scorign domain in sequenc--> idea is that we might be able to detect that a sequence is apart of a multi-domain familiy b/c it contains multiple weakly-scoriong domains. On the other hand, if the target sequence is bad (meaning, it contains a set of identical internal repeats) and  one of its repeats gives a weak hit to query profile, all repeats will sum and make sequence score look significant even if that is not the case




In [ ]:
#handle_arguments is for taking user input for where to output file
def handle_arguments():
  #helps user see what they need to input
  description = '''This script parses the tcDoms domtblout file,
        selects the top scoring annotation for each protein listed in the
        file, and then outputs a csv of the parsed and down-selected data.

        Example usage: ./best_tcDoms.py -f domtblout.tab annotations.csv
        '''
  #outputs decription
  parser = argparse.ArgumentParser(description = description)
  #add input and output requirements; help gives user more info abt what to put
  #input is filepath
  #output is annotates proteinsin a csv
  parser.add_Argument('input' , type=str, help = 'Input domtblout file' )
  parser.add_Argument('output', type = str, help = 'Output file')
  return parser

def main():
  #gets parser obj from handle_arguments
  parser = handle_arguments()
  #parses arguments into
  args = parser.parse_args()
  #make sure output path is valid
  output_path = args.output
  #checks weather given path is an existing file or not
  if os.path.isfile(output_path):
     raise FileExistsError('A file by the name of {} already exists.'.format(output_path))

  #parse tcDoms Domtblout file
  #outputs get placed into a dataframe
  print("Parsing Domblout file", flush = True)
  df =pd.read_csv(
      args.input,
      engine = 'python',
      sep ='\s+',
      skiprows = [0,1,2],
      skipfooter = 10,
      usecols = [0,3,6,7],
      names = ['Target_name', 'query_name', 'E-value', 'Score']
  )
  #taking the best result for each query
  print("Selecting the best annotation for each protein", flush = True)
  #new dataframe for best annoatation
  #we take the original df and group by the query name
  #(same query_name correlates to the same protein)
  #we then select the index that has the max score within this group
  #this is then assgined to best_annot_df
  #ultimately, this profuced a Df of unique proteins what had the highest
  #scores within groups of the same protein
  df['score'] = df['score'].astype(float)
  best_annot_df = df.iloc[df.groupby(['query_name']).score.idxmax()]
  #reset index makes the indexes so that they are sequential
  #instead of keeping the old dfs orfinal index (out-of-order now in new df)
  #drop = true ensures a new column is not made for the old index
  #inplace = True, this modifies the current df instead of creating a new one
  best_annot_df.reset_index(drop = True, inplace = True)

  #save results to csv
  best_annot_df.to_csv(args.output, index = False)
  print('Finished')
  if __name__ == "__main__":
    main()






